# 質点系のシミュレーション

吉田勝俊（宇都宮大学）

## 参考情報

- [Pythonで運動方程式を解く(odeint) - Qiita](https://qiita.com/binaryneutronstar/items/ad5efa27fd626826846f)
- [Numpyの参照、抽出、結合 - Qiita](https://qiita.com/supersaiakujin/items/d63c73bb7b5aac43898a)

In [ ]:
import numpy as np                              #数値計算ライブラリ
from numpy.linalg import norm                   #ベクトルのノルム
from scipy.integrate import odeint              #常微分方程式ライブラリ
import matplotlib.pyplot as plt                 #描画ライブラリ

## 3質点系の状態方程式

結果の見やすさのため2次元で実装します．

In [ ]:
m1, m2, m3 = 1, 1.2, 1.5  #質量
K = 1 #ばね定数

def State_Equation(XX, t, masses, spring_constant):
  '''
  3質点系の状態方程式
  '''
  m1, m2, m3 = masses   #質量
  K = spring_constant   #ばね定数

  ### 状態ベクトルの分割
  # 位置ベクトル達
  x1 = XX[0:2]    #XX[0]〜XX[1]　終端の添字は+1で指定する仕様
  x2 = XX[2:4]    #XX[2]〜XX[3]
  x3 = XX[4:6]    #XX[4]〜XX[5]
  # 速度ベクトル達
  v1 = XX[6:8]    #XX[6]〜XX[7]　終端の添字は+1で指定する仕様
  v2 = XX[8:10]   #XX[8]〜XX[9]
  v3 = XX[10:12]  #XX[10]〜XX[11]

  ### 外力/質量 = 重力加速度ベクトル
  fk_by_mk = np.array([0,-9.8])

  ### 内力の計算
  f12 = K*(x2-x1)
  f23 = K*(x3-x2)
  f31 = K*(x1-x3)
  f21 = -f12
  f32 = -f23
  f13 = -f31
  
  ### 右辺F(X)の値
  #np.arrayは横ベクトルなので，水平に積む
  FX = np.hstack((
      v1,
      v2,
      v3,
      fk_by_mk + (f12 + f13)/m1,
      fk_by_mk + (f21 + f23)/m2,
      fk_by_mk + (f31 + f32)/m3,
  ))

  return FX

シミュレーション用のユーザ関数

In [ ]:
def Simulate(K):
  '''
  各質点の運動と重心運動を計算する
  '''
  ts = np.linspace(0,5,200) #時間軸を表す数列（0〜5秒を200等分）

  XX0 = np.array([
     0,0,  0,0,  0,0, #初期位置＝原点 
    -5,20, 2,25, 5,20 #初速度
  ])
  masses = [1,1.2,1.5]  #質量
  spring_constant = K   #ばね定数

  ###状態方程式（微分方程式）を解く
  XX = odeint(State_Equation, XX0, ts, 
              args=(masses, spring_constant))
  
  x1t = XX[:,0:2]   #質点m1の解
  x2t = XX[:,2:4]   #質点m2の解
  x3t = XX[:,4:6]   #質点m3の解

  ###重心の軌跡G(t)を求める
  M = sum(masses)     #全質量
  m1, m2, m3 = masses #各質量
  Gt = (m1*x1t + m2*x2t + m3*x3t)/M

  return (x1t, x2t, x3t, Gt)

## 運動の軌跡

内力が無い場合

In [ ]:
x1t0, x2t0, x3t0, Gt0 = Simulate(K=0)

plt.plot(x1t0[:,0], x1t0[:,1], color='C1', label='m1(K=0)')
plt.plot(x2t0[:,0], x2t0[:,1], color='C2', label='m2(K=0)')
plt.plot(x3t0[:,0], x3t0[:,1], color='C3', label='m3(K=0)')
plt.plot(Gt0[:,0], Gt0[:,1], color='C4', label='center(K=0)')
plt.legend()

内力が有る場合（ばねによる）

In [ ]:
x1t, x2t, x3t, Gt = Simulate(K=8)

plt.plot(x1t[:,0], x1t[:,1], color='C1', label='m1')
plt.plot(x2t[:,0], x2t[:,1], color='C2', label='m2')
plt.plot(x3t[:,0], x3t[:,1], color='C3', label='m3')
plt.plot(Gt[:,0],  Gt[:,1], color='C4', label='center')
plt.legend()

重心運動の軌跡（内力有無を同じグラフに）

In [ ]:
plt.plot(Gt0[:,0], Gt0[:,1], 'o', label='Free')
plt.plot(Gt[:,0], Gt[:,1], '-', label='Spring')
plt.legend()

- 質点運動は全く異なったのに，重心運動は理論通り一致しました．